In [ ]:
import numpy as np
from Crypto.Util import number
import Crypto.Random as Random

In [ ]:
CATEGORIES = [0, 1, 2]

class R:
    def __init__(self, data, epsilon, n, l):
        self.data = data
        self.epsilon = epsilon
        self.n = n
        self.l = l
    
    def setup(self):
        t = self.data
        print("secret input: ", t)
        mu_array = []
        if t == 0:
            mu_array = [0]*self.l + [1]*(self.n-self.l)
        elif t == 1:
            mu_array = [0]*(self.n-self.l) + [1]*self.l
        else:
            assert False, "error t"
        mu_array = np.random.permutation(mu_array).tolist()
        mu_array.append(1-t)
        self.mu_array = mu_array
        print("mu array: ", mu_array)
        
    def set_pub_key(self, pub_key):
        self.pub_key = pub_key
    
    def set_group(self, group):
        self.group = group
        
    def step2(self, g_a, g_b, g_ab):
        g, h = self.pub_key
        p, q = self.group
        w_array = []
        y_array = []
        v_array = []
        
        for i in range(1, self.n+2):
            r = number.getRandomRange(1, q-1)
            s = number.getRandomRange(1, q-1)
            w = pow(g, r, q) * pow(g_a, s, q) % q
            v = pow(g_b, r, q) * pow(g_ab * pow(g, i-1, q) % q, s, q) % q
            y = pow(g, self.mu_array[i-1], q) * pow(h, v, q) % q
            w_array.append(w)
            y_array.append(y)
            v_array.append(v)

        self.w_array = w_array
        self.y_array = y_array
        self.v_array = v_array

        return v_array, w_array, y_array
    
    def AKEncBool1(self):
        g, h = self.pub_key
        p, q = self.group
        b_array = []
        c_array = []
        s_array = []
        random_w_array = []
        for y, mu in zip(self.y_array, self.mu_array):
            random_w = number.getRandomRange(1, q-1)

            if mu == 0:
                c_1 = number.getRandomRange(1, q-1)
                c_array.append(c_1)
                s_1 = number.getRandomRange(1, q-1)
                s_array.append(s_1)
                b_0 = pow(h, random_w, q)
                g_1_inv = pow(pow(g, 1, q), -1, q)
                deno = pow(y * g_1_inv % q, c_1, q)
                b_1 = pow(h, s_1, q) * pow(deno, -1, q) % q
            elif mu == 1:
                c_0 = number.getRandomRange(1, q-1)
                c_array.append(c_0)
                s_0 = number.getRandomRange(1, q-1)
                s_array.append(s_0)
                b_1 = pow(h, random_w, q)
                g_0_inv = pow(pow(g, 0, q), -1, q)
                deno = pow(y * g_0_inv % q, c_0, q)
                b_0 = pow(h, s_0, q) * pow(deno, -1, q) % q
            else:
                assert False, "error mu"

            b_array.append((b_0, b_1))
            random_w_array.append(random_w)

        self.b_array = b_array
        self.s_array = s_array
        self.c_array = c_array
        self.random_w_array = random_w_array
        
        return b_array
    
    def AKEncBool3(self, ak_enc_bool_x_array):
        p, q = self.group
        c_array = []
        s_array = []
        for x,c,s,mu,v,random_w in zip(ak_enc_bool_x_array, self.c_array, self.s_array, self.mu_array, self.v_array, self.random_w_array):
            if mu == 0:
                c_0 = (x - c)
                c_1 = c
                s_0 = ((v * c_0) + random_w)
                s_1 = s
            elif mu == 1:
                c_0 = c
                c_1 = (x - c)
                s_0 = s
                s_1 = ((v * c_1) + random_w)
            else:
                assert False, "error mu"

            c_array.append((c_0, c_1))
            s_array.append((s_0, s_1))
        return c_array, s_array
    
class I:
    def __init__(self, epsilon, n, l):
        self.epsilon = epsilon
        self.n = n
        self.l = l
    
    def setup(self, security):
        q = number.getPrime(2 * security, Random.new().read)
        p = 2*q + 1
        
        g = number.getRandomRange(1, q-1)
        h = number.getRandomRange(1, q-1)
        
        self.p = p
        self.q = q
        self.g = g
        self.h = h
        
        self.sigma = np.random.randint(1, self.n+1)
        print("sigma: ", self.sigma)
        
        self.pub_key = (g, h)
        
    def step1(self):
        a = number.getRandomRange(1, self.q-1)
        b = number.getRandomRange(1, self.q-1)
        self.a = a
        self.b = b
        
        g_a = pow(self.g, a, self.q)
        g_b = pow(self.g, b, self.q)
        g_ab = pow(self.g, a * b - self.sigma + 1, self.q)

        return g_a, g_b, g_ab
        
    def step3(self, w_array, y_array):
        v_sigma = pow(w_array[self.sigma-1], self.b, self.q)
        g_mu_sigma = y_array[self.sigma-1] * pow(pow(self.h, v_sigma, self.q), -1, self.q) % self.q
        secret_output = 0
        if g_mu_sigma == 1.0:
            secret_output = 0
        else:
            secret_output = 1
        print("secret output: ", secret_output, "g^{mu_sigma}: ", g_mu_sigma)
        return secret_output
    
    def AKEncBool2(self, num):
        self.ak_enc_bool_x_array = []
        for _ in range(num):
            self.ak_enc_bool_x_array.append(number.getRandomRange(1, self.q-1))
        return self.ak_enc_bool_x_array

    def AKEncBool4(self, s_array, c_array, b_array, y_array):
        print("####### verification #######")
        for s,c,b,y,x in zip(s_array, c_array, b_array, y_array, self.ak_enc_bool_x_array):
            for i in [0,1]:
                if pow(self.h, s[i], self.q) != b[i] * pow(y * pow(pow(self.g, i, self.q), -1, self.q) % self.q, c[i], self.q) % self.q:
                    print("AKEncBool False.")
                    return False
            if x != c[0] + c[1]:
                print("AKEncBool False.")
                return False
        print("AKEncBool OK.")
        return True 
    
    def AKLin(self, y_array, v_array):
        commitment = 1
        for i in range(len(y_array)-1):
            commitment *= y_array[i] % self.q
        commitment = commitment * pow(y_array[len(y_array)-1], 2 * self.l - self.n, self.q) % self.q
        
        sum = 0
        for i in range(len(v_array)-1):
            sum += v_array[i]
        sum += (2 * self.l - self.n) * v_array[len(v_array)-1]
        
        if commitment == pow(self.g, self.l, self.q) * pow(self.h, sum, self.q) % self.q:
            print("AKLin OK.")
            return True
        else:
            print("AKLin False.")
            return False

In [ ]:
sender = R(0, 1.0, 3, 2)
sender.setup()

receiver = I(1.0, 3, 2)
receiver.setup(80)

pub_key = receiver.pub_key
sender.set_pub_key(pub_key)
group = (receiver.p, receiver.q)
sender.set_group(group)

g_a, g_b, g_ab = receiver.step1()
v_array, w_array, y_array = sender.step2(g_a, g_b, g_ab)
x = receiver.step3(w_array, y_array)

b_array = sender.AKEncBool1()
ak_enc_bool_x_array = receiver.AKEncBool2(len(y_array))
c_array, s_array = sender.AKEncBool3(ak_enc_bool_x_array)
receiver.AKEncBool4(s_array, c_array, b_array, y_array)
receiver.AKLin(y_array, v_array)

In [ ]:
def lipmaa(val):
    for i in range(0,100000):
        for j in range(0,1000):
            for k in range(0,100):
                for l in range(0,10):
                    if (((i*i) + (j*j) + (k*k) + (l*l)) == val): return (i,j,k,l)
    return(0)

def genFourRandom(q):
    r_0 = number.getRandomRange(1, q-1)
    r_1 = number.getRandomRange(1, q-1)
    r_2 = number.getRandomRange(1, q-1)
    r_3 = number.getRandomRange(1, q-1)
    return r_0, r_1, r_2, r_3

def createCommitments(x, r_0, r_1, r_2, r_3, g, h, q):
    x_0, x_1, x_2, x_3 = lipmaa(x)
    c_0 = pow(g, x_0**2, q) * pow(h, r_0, q) % q
    c_1 = pow(g, x_1**2, q) * pow(h, r_1, q) % q
    c_2 = pow(g, x_2**2, q) * pow(h, r_2, q) % q
    c_3 = pow(g, x_3**2, q) * pow(h, r_3, q) % q
    return c_0, c_1, c_2, c_3

In [ ]:
security = 80
q = number.getPrime(2 * security, Random.new().read)
g = number.getRandomRange(1, q-1)
h = number.getRandomRange(1, q-1)

# prove
x = 100
a = 4
b = 8
r_0, r_1, r_2, r_3 = genFourRandom(q)
c_0, c_1, c_2, c_3 = createCommitments(x, r_0, r_1, r_2, r_3, g, h, q)
r = r_0 + r_1 + r_2 + r_3
secret = pow(g, x, q) * pow(h, r, q) % q

# verify
c = c_0 * c_1 * c_2 * c_3 % q
print(secret == c)

# prove
c1 = pow(g, (b-x), q) * pow(h, -r, q) % q
p1 =(pow(g,b,q) * pow(c1,-1,q)) % q
print(secret == p1)

c2 = pow(g, (x-a), q) * pow(h, r, q) % q
# verify
p2 = c2 * pow(g, a, q) % q
print(secret == p2)

In [ ]:
p1

In [ ]:
secret